In [1]:
import requests
import pandas as pd

In [2]:
# Settings
disco_base_url = 'https://disco.eduvpn.org/'
organisation = 'SURFnet bv'
server = 'Demo'

In [3]:
org_list_url = disco_base_url  + "organization_list.json"

In [4]:
org_list_response = requests.get(org_list_url)

In [5]:
if not org_list_response:
    raise Exception(f"failure getting {org_list_url}")

In [6]:
organization_list = pd.DataFrame(org_list_response.json()['organization_list'])

In [7]:
organization_list[:5]

,display_name,org_id,server_list,keyword_list
0,{'da': 'International Business College [TEST]'...,https://wayf.ibc.dk/simplesaml/saml2/idp/metad...,aHR0cHM6Ly93YXlmLmliYy5kay9zaW1wbGVzYW1sL3NhbW...,NaN
1,"{'da': 'Erhvervsakademi Aarhus', 'en': 'Busine...",http://adfs.eaaa.dk/adfs/services/trust,aHR0cDovL2FkZnMuZWFhYS5kay9hZGZzL3NlcnZpY2VzL3...,NaN
2,"{'da': 'KMD DSPARE3', 'en': 'KMD DSPARE3'}",http://dans-support-04.dans-idp-dev01.northeur...,aHR0cDovL2RhbnMtc3VwcG9ydC0wNC5kYW5zLWlkcC1kZX...,NaN
3,"{'da': 'KMD DSUP_PATCH1', 'en': 'KMD DSUP_PATC...",https://dans-idp.kmd.dk:7080,aHR0cHM6Ly9kYW5zLWlkcC5rbWQuZGs6NzA4MA.json,NaN
4,"{'da': 'IT-Supportcentret (ITS) [ADFS]', 'en':...",http://fs4.supportcenter.dk/adfs/services/trust,aHR0cDovL2ZzNC5zdXBwb3J0Y2VudGVyLmRrL2FkZnMvc2...,NaN


In [8]:
def pandas_expand(df: pd.DataFrame, column: str):
    expanded = pd.DataFrame(list(df[column])).add_prefix(column + '_')
    return pd.concat([expanded, df.drop(columns=column)], axis=1)

In [9]:
org_list_expanded = pandas_expand(organization_list, 'display_name')

In [10]:
surfnet = org_list_expanded[org_list_expanded['display_name_en'] == organisation]
server_list_url = disco_base_url + surfnet['server_list'].iloc[0]

In [11]:
server_list_response = requests.get(server_list_url)

In [12]:
if not server_list_response:
    raise Exception(f"failure getting {server_list_url}")

In [13]:
server_list = pd.DataFrame(server_list_response.json()['server_list'])
server_list_exp = pandas_expand(server_list, 'display_name')

In [14]:
server_list_exp

,display_name_da-DK,display_name_en-US,display_name_nb-NO,display_name_nl-NL,display_name_en,base_url,peer_list
0,Holland,The Netherlands,Nederland,Nederland,NaN,https://nl.eduvpn.org/,"[{'base_url': 'https://eduvpn.rash.al/', 'disp..."
1,NaN,NaN,NaN,NaN,Demo,https://demo.eduvpn.nl/,NaN
2,NaN,NaN,NaN,NaN,SURFnet,https://surfnet.eduvpn.nl/,NaN


In [15]:
demo_server = server_list_exp[server_list_exp['display_name_en'] == 'Demo']

In [16]:
demo_base_url = demo_server['base_url'].iloc[0]

In [17]:
demo_base_url

'https://demo.eduvpn.nl/'